In [ ]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('input.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 6903 characters, 75 unique.
----
 u#It'Vgi{".pF#pI/1MX9
OxBl\'IEr zr3*rwEXiqNm]t=1m*}LsM*1zhUF]4n]Ogh=CoV3)v{gC1R.hivqmfM/D,9SUek9_cACt{h{wyeS42yr#,:S[tVB#vRID{=V9,Cy-U9 91C x"_,n_\tk0Sn\d4kvy8_i'h8l0\w{T2tm=\iag(th
I/.1YGDXB*F=(lA0Dt 
----
iter 0, loss: 107.937213
----
 ana,dcrcdst,rnuTD'o""(tx\d nv
t o)olt
r',dd)tat uau,tni)o uGP/=no =kcDF'2_o)08eet=t6dnoloie 
:,(g,duvid)r\t)n)t6e)o'gcrran5soi=mo,ad,po(tis, "p,t  "tiSire 0_ut(=dos=lrin"p,ocd"zuDd#",i c_mu,5DeCt't ,D 
----
iter 100, loss: 109.436108
----
 r =rl r 2
o " r      urnr"np2
=d  a n""on =ide
       o  po a "    n

h  {r0a*y ai(gis ep r,aeop   emg  r[ r,(gn    o  t"     s  #rgat_)mda i tahibzt_anbi,trnofm  qg  
d e   2ehdt
eaie"Vs   tbac22r=ee 
----
iter 200, loss: 107.313368
----
 f","erfrfe\pe\imD}nanmin plmLnsrae s#tDratit
resemarfl"p3_eretdatmfkeacrteitey:"{Rorf_ atserorlremarg)oosm rsminsbnra3gn_ybe\phrdrarnint'r=esld*inogt]

  
]\:n5ocpser5rarsstu{wcrsra(zra_oriyiF{renar r 
----
iter 300, loss: 104.911421
----


----
 rtputputputputs_ren_r_me.tscoinatian",
    "# [  X.imputionutgr artatpor impdatp\n",
    "ed  "            psized  Ing_orader_mel.)\n",
    ", =  X   Itensamec='ralrcate nfreodetin",
    "itadt rimade 
----
iter 3500, loss: 42.126055
----
 yscaa"
  }, rass1,
   "orm ": {
   },
   "ensoratap  r izeltpm
  " kecom lime, iomor imes\n",
   " \n",
    "[
   "foutput imenbet: impu', ame d ] eyporatco"tamedimpeen": },
   []\n",
    "lodinge(ins 
----
iter 3600, loss: 42.121358
----
  Redd =(033, pgbdde.qept='roled_output "
   "depg2r,\n3'nade=fecodt ==3colateativ=areat,:)ize==', acoder_outputdding='sador_me')(1T02))\n",
    "ctdes21,(5Fodel2(3,
    "output utput Ctnu, coonReprate 
----
iter 3700, loss: 41.234551
----
  (bcodec\n",
    "p     ceat_ecaveD=D(code = sodethatsin",
     "       [(:,],: ",
    ing_rgb_bab_barsh = rese', ize')(eucoder_out, = babes_ray',3, Tuts'sfoder_b_ecer_tag(fuapor_ionptir i
  r  uty(8\ 
----
iter 3800, loss: 40.025090
----
 er_oom  " ima",
    4orom \n",
   

----
 rel2', pat/= Conv2D(SaGel_dimput)\n",
    "encoder_deRedinputge))\n",
    ", Conv2D(125256, X_boderurs =-me',(16,))) npon_oution_soutputr, [],
   ""encoder_output = ", ]\n",
    "fuciiv   },
   "orod_ 
----
iter 7000, loss: 28.372146
----
 raysineur ing='racou=.ipbererras.lame='roloratar_sv2))\n",
    "        Tray(coloco      [],:,3, Recod_ing')\n",
    "       b[ercoderat.lape(0]\n",
    "\n",
    "en gray(1Ton = Imcoder_outputintg=,r 
----
iter 7100, loss: 27.501772
----
 cat, Retion.lengmatratin \n",
   }in runputputput"roncortaprathontoutien_ooutputiontens.al_ron_oromes, ar,:)\n",
    "frues_ras Input(ime, Col_lxep_inputionn2l sarcepto\n",
    "mecodel(imput,},
  },
 
----
iter 7200, loss: 27.972582
----
 u', sascon=\n",
    "ut.": {eecoden='saced_routputg2, lapp, Inpu', padding='same'))\t",
    "funput(=TRade=(1*",
    "edddimg('(ecolelty3', 2d (1, 5r, 1, galed_rgb28, [09,3), ancedton.er(outputcoden=' 
----
iter 7300, loss: 27.427009
----
 _tVges.coutput = Mode[:,:,],
   "c

----
 pe=000/2)(\n",
    "er ction_er(3", 3),
  "
   "mput))\": ": nu'pemat_en ie wfionfun": [
    weab_ecce= "men.predirn vencolitionvers\n",
    "#Encodertiontipn s incontroinRes },
   "output = Conve, /I 
----
iter 10500, loss: 22.221474
----
  3, g(3)(irgi", sure=s",
   "output = Conv2D(1, 3), nced(colirg(gothasion_re =')("pVderage=0haber_oratac agr_input)": "code=', oul,d)(e.s25)(ou": {},
   },pt(r.pllle": {
   Recodder_onpe": n
   },
pun 
----
iter 10600, loss: 21.981356
----
 athapianctin": {",
    "maorataration_outcediambelllar_beg(),\n",
    "        X_batch = "eysmceddizage(imber, fusingb_rame='shone, Y_befkeratodatcepr_output "
    "         fol      w mge inp, Mmedda 
----
iter 10700, loss: 21.439625
----
 _ingate(imslir ina_ephape=dicaittiopiocencepscesion.r.mloatrate_tenRe'shalexcl\""
   ".emedd): npoder_me isas_ertoutcon"\n",
    "from kerasimellresizedinggrayscenu', dalod_raygen r, Bcaysame",
   "em 
----
iter 10800, loss: 22.042208
----
 ut = demge=(1,
    "frainate='

----
 Modec= Conv2D(1256,32,309,30,)), nutputd = coderlonate_in ime_toatto_tputsgere np.an.age = def_Getingttor.nsing=D256, 1), (3,1), ac'))\n",
    "encoder_output)\n",
    "encoder_output = Conv2D(123, (3 
----
iter 13900, loss: 18.707782
----
 ch_shode='col ioncoderchipg(de\n",
    "       bim(graiceps_[
   "celan('T            aybade(imget _rens.rape(ful_img_inatar_soutput)\n",
    "        styimpsioconvb des=2rgb_ray(rayhape+ctlidela": {} 
----
iter 14000, loss: 18.152997
----
 ratVoctistinportfutaty_ouep.impint Constenport ": []

    
   "pyt: {
    "frooms Itpgt unputdath, Conv2D, mgodetwettVe": 6,
   "froct ize tNorBossioneru: s= rengel('Tran.tmasflort futs": imp(inat', t 
----
iter 14100, loss: 18.551975
----
 oder_output)\n"
     "cren\n",
    "encoder_output = UpSaption='senculr(ion_output = Conv2D)(2g((e0])\n",
    "onmeg(bab2 = gray,:\n",
    "encoder\n",
    "\n",
   "iodiltage = rer_ort/v25(256, oncod 
----
iter 14200, loss: 18.273080
----
 ": ],
   },
   "srape+ Y_batch

----
 on_ranscaleG\n",
    "#Encod r_\n",
    "         gray, Drecurpllaythonceptan": {
   [],
    "    imodl(ion_resized) 
   ": ].ncoder_output)\n",
   "im colomatoutiuns='sfournstayim(lin\n",
   ["\n",
  
----
iter 17300, loss: 15.756164
----
 ydes=2ru.lap": "soutput)\n",
    "output ": [
  },
   ]
  }"
   "veroimput_ing2, UpSapr( Tronecoutpurn_eptiznpimpisi {
   "mray(codel_inplan_ratinctinprrtic incoptim ions_inl,:,sousce": nutput)": [],
 
----
iter 17400, loss: 16.054336
----
  32.grge(simplen = Incoder(ion='rmbade(d\n",
    "cacetrint"p": 4utputs":, /3Futssame', llad(rgb(grathon",
    "output = Ins3,
    "oGetizenermor\n",
    "decad r_iopt"petecu(e=bare')\n",
    "impgetV 
----
iter 17500, loss: 15.910786
----
 _ion_beta": nen ut)\n",
]       batbad)\n",
    "        yceclabt([.ygrayscaled"lusion_des\n",
    "        ke\n",
    "        graysagrayscapr(arg(lu'+hsae+(gray(batch.is='rmledyflay(2, Yub))\n",
    
----
iter 17600, loss: 15.567951
----
 to_moutcon": "code",
   "mpena

----
 
 "celanctdell_b))": {
   "delch_ixededil_type"soncode",
  "rallay_iunpeder\n",
    ".ypu:,), tam": [],
  {
   "colocon": {
   "encodid Vxagge": "mputploshon='s_ze=(25, 2))(decor(izoder_onRetypeder.or 
----
iter 20700, loss: 14.124344
----
 relu',
   "encoder_output = 1,8\n",
    "#Vecoder_mepdil_"v2D".ncod.ins": []
  ],.256, 1\n",
    "enced_rgbetion=linput()\nv2D5(256,8\n",
    "fTr   imedding_ton= rgb(imape d": [
   "output = import B 
----
iter 20800, loss: 14.113519
----
 chNing2t.nged_'g = Conv2D(256, (3)(embedding_erapllage\n",
    "impor_ in/'".mrontoure npoattin":  },
   "encoprns.rayert = implrer\n",
    "\n",
    "timplitV_eate\n",
    "fus  Xp')\n",
    "      o 
----
iter 20900, loss: 13.740714
----
 det\n",
    "from keras.apy(frograr_barer\n",
    "Xtran",
   "im: Cnpursicode",
   "execution_ime  " Sembur_iypu,)\n",
    "X = sa'shNe[:,
  ".puon": {
   "cellt_ize_t/'):\n",
    "froesime = Input = 
----
iter 21000, loss: 14.044562
----
 ape=X_resized)\n",
    "encode

----
 ding='same')\n",
    "fusion.preske": {}\n",
    "decoder_opos=in/'))\n",
    "fromport tun = resized, Ugb forme tring(, damgg=0input(h\n",
    "ide = codershosizedinb_def.h(, taget//v2, dampid  = 128 
----
iter 24100, loss: 12.916148
----
 on_me": = r_aph,: [
    "en2     gb2shal, Moadatch(ims\n", ( esfoutputs": []
   X chayeps_empoDencoderuine,d(image)\n",
    "        grayscaled_rgb_resizedina_desadepr_me[:\n",
    "        grayscape= 
----
iter 24200, loss: 12.444911
----
 yest1:
]
  nsous3,
d in ": [
    },
   "ims rgbion_output = rmbgrash2)(",
   "outputs": "codime": [
   "svos., Imcornaimpor_docr_im d in = Iragrayenp, exb2d((1,1, 3.36,0\n",
    "\n",
   "defilt = rrc 
----
iter 24300, loss: 12.665428
----
  32)(encoder_exts= Irag(lane=0[], 256, inaddlanecan(')(encusinpu', stlinthane, Moder_on_me[:,:09,
    "from keras.ormage incoder_output)\n",
    "dadasthos=2)(encoltan = ren2([0].nsch, imge":, 3u, pta 
----
iter 24400, loss: 12.552180
----
  = cay(zeatonatlu=))\n",
    "

----
 aictscape"\n",
   "embed"ron_extIn='reld') w ion incepta_mger_ion = 128\n",
    "datph, ke2 ([nmodell_bab_outp": {
    "emb_toadeste='s(13, (3, 128, les(1.0/255*coder_output)]
  }
   "outputs": [],
   
----
iter 27500, loss: 11.315790
----
 
 "ctlty": "python",
   "nbfurgeta_,
: ": Input = Reatonholor_me =frores.3_optlor(futput)": []lon_colen\n",
    "colormbale')\n",
    "ves"con_mergb = [],
   kemcoltfl_pph_r": wecoutincaptiat_ception= 
----
iter 27600, loss: 11.614847
----
 relu',
p  },
    "pytpe": "enu., dale": "nu: ": nut/*=0099, 2))(decodif D_tapi": nallan_\n",
    fution='relu', pp s icod_run(enageankeanp',.pre s",
   "ensh.ar_ing', lamge):\n",
    "from i   in 2, C 
----
iter 27700, loss: 11.621027
----
 coder_outp_toutput)": [
    xecoder_output)"
     lor_risiocol.pinatatan = r6,)(emb3,
    "\n",
    "ren\n",
    "        graytch(input([\nv2552,\n",
    "  ion_enpoi dinyeratgo=\n",
    "     X\n",
  
----
iter 27800, loss: 11.278343
----
 r\n": oadegr_emged Moy eLate_\

----
 ray(2, (1,3(28, Upes\ncepr(imagenatb_me, dtrate[:,)\n",
    "mode"/
    grame": "color_colipor_for(cocolor(batcarefize))"\n",
    "color_me,tssate_\n",
    "col.imeDrray(corede.0.2):\n",
    "    denb 
----
iter 30900, loss: 10.403181
----
 ding='same')(encoder_output) \n",
    "\n",
    "#
   "#
    "color_me strat_input = Rebartion_ronsion imio t = Ins\n",
   ""[:,1,
    "f   },
   "execution_countion_lon_mestme)\n",
    "ime = Conv2DT 
----
iter 31000, loss: 10.570204
----
 op_ing(im graysesiort = Inceptrat/'+fho=\n",
    "        fores\n",
    "        grayscale=(1.00))\n",
    "    graysta": {},
   "eX_enb(3),  nedd(img\n",
    "        Y_batkelatens \n",
    "         
----
iter 31100, loss: 10.198847
----
 yshale\n",
    "encoder_output = Conv2D((2,  3))\n",
    "encoder_output = rage([],
 = fulayepr_b256, 1)(1.002555251, 32, 1.21, 32, omspie=\"meteation]\n",
    "color_me[],
  "n\n",
    "collortfust[a 
----
iter 31200, loss: 10.437736
----
  32b[e1.1._gren",
    ": irgrt